# _Fairness_ and _Interpretability_ in the COMPAS dataset

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).

**Fairness and interpretability are two important considerations in machine learning, and they are closely related. Fairness refers to the absence of bias or discrimination in the ML models and their outputs, while interpretability refers to the ability to understand and explain how the models make their predictions.**

**In some cases, improving the interpretability of a model can help to identify and correct sources of bias or discrimination, which in turn can improve the fairness of the model. For example, if a model is found to be biased against a particular group of people, it may be necessary to examine the features that the model is using to make its predictions and determine whether any of these features are unfairly influencing the model's output.** 

**By improving the interpretability of the model, it may be possible to identify these problematic features and make changes to reduce bias and discrimination.**

**Conversely, fairness can also be used as a metric to evaluate the interpretability of a model. A more interpretable model may be easier to assess for bias and discrimination, as it is easier to understand how the model is making its predictions. This can be particularly important in high-stakes applications, such as healthcare or criminal justice, where the consequences of a biased or unfair model can be significant.**

**Therefore, fairness and interpretability are closely linked, and it is important to consider both aspects when developing ML models. In this notebook, we will try to work with both these principles using the `COMPAS` dataset.**

**_Correctional Offender Management Profiling for Alternative Sanctions_ (`COMPAS`) is a case management and decision support tool developed and owned by Northpointe (now [Equivant](https://www.equivant.com/)) used by USA courts to assess the likelihood of a defendant becoming a recidivist.**

**The `COMPAS` software uses an algorithm to assess potential recidivism risk. Northpointe created risk scales for general and *violent recidivism*, and for *pretrial misconduct*.**

![compas](https://cdn.psychologytoday.com/sites/default/files/styles/article-inline-half/public/field_blog_entry_images/2018-01/compas.jpg?itok=H4USxzQ3)

## **Ethical Problems**

- **A general critique of the use of proprietary software such as COMPAS is that since the algorithms it uses are *trade secrets*, they cannot be examined by the public and affected parties which may be a violation of due process.**
- **Another general criticism of machine-learning based algorithms is since they are data-dependent if the data are biased, the software will likely yield biased results.**

**In this notebook, we will be using *COMPAS Recidivism Racial Bias* (available on [Kaggle](https://www.kaggle.com/datasets/danofer/compass?select=cox-violent-parsed_filt.csv)) to explore certain tools related to ML Fairness and Explainable AI (XAI). In our example, we will be using the parsed data from this dataset (`cox-violent-parsed_filt.csv`), which contains 18316 samples.**

**To create a classifier from scratch, we first need to get rid of the labels (scores and categories) that the original algorithm produced. For better performance purposes, we are also excluding features that have more than $10\%$ of their total bulk as missing values ("`NaN`").** 

**This resulted in a final sample size of $17019$. In the end, we are left with a dataset containing 12 features + the label. For simplicity, we are merging the "*Low*" and "*Medium*" labels, to turn this classification task into a binary problem (Fairness analyses are simpler in these cases). "*High Risk*" samples represent only $25\%$ of our dataset, and these are exactly the cases we want to better distinguish.**

In [5]:
import pandas as pd
df = pd.read_csv("data/COMPAS.csv")

for column in df.columns:
    nan = df[column].isna().sum()
    if round((nan / len(df[column])) * 100, 2) > 10.0:
        print(f'Feature {column} : {round((nan / len(df[column])) * 100, 2)}% is NaN values.')

def turn_to_binary(score):
    if score == 'Low' or score == 'Medium':
        return 1
    else:
        return 0

df['label'] = df['score_text'].apply(turn_to_binary)

df = df[['sex', 'age_cat', 'race',
        'juv_fel_count', 'juv_misd_count',
        'juv_other_count', 'priors_count',
        'days_b_screening_arrest', 'c_days_from_compas',
        'c_charge_degree', 'is_recid', 'is_violent_recid',
        'label']].dropna()
                
with pd.option_context('display.max_columns', None):                     
    display(df)

Feature id : 39.94% is NaN values.
Feature r_charge_degree : 54.05% is NaN values.
Feature r_days_from_arrest : 65.28% is NaN values.
Feature r_offense_date : 54.05% is NaN values.
Feature r_charge_desc : 54.81% is NaN values.
Feature r_jail_in : 65.28% is NaN values.
Feature violent_recid : 100.0% is NaN values.
Feature vr_charge_degree : 92.69% is NaN values.
Feature vr_offense_date : 92.69% is NaN values.
Feature vr_charge_desc : 92.69% is NaN values.


,sex,age_cat,race,juv_fel_count,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,c_charge_degree,is_recid,is_violent_recid,label
0,Male,Greater than 45,Other,0,0,0,0,-1.0,1.0,(F3),0,0,1
1,Male,Greater than 45,Other,0,0,0,0,-1.0,1.0,(F3),0,0,1
3,Male,25 - 45,African-American,0,0,0,0,-1.0,1.0,(F3),1,1,1
4,Male,Less than 25,African-American,0,0,1,4,-1.0,1.0,(F3),1,0,1
5,Male,Less than 25,African-American,0,0,1,4,-1.0,1.0,(F3),1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18311,Female,25 - 45,African-American,0,0,0,5,-1.0,1.0,(M1),0,0,1
18312,Male,Greater than 45,Other,0,0,0,0,-1.0,1.0,(F2),0,0,1
18313,Female,25 - 45,African-American,0,0,0,3,-1.0,1.0,(M1),0,0,1
18314,Female,Less than 25,Hispanic,0,0,0,2,-2.0,2.0,(F3),1,0,1


**Now, let us try to se how our target is related to the sensitive atributes of our dataset (`Age`, `Race`, and `Sex`).**

In [6]:
high_risk = []
low_risk = []

for element in list(df['sex'].unique()):
    a = df[df['sex'] == element]['label'].value_counts()[0]
    b = df[df['sex'] == element]['label'].value_counts()[1]
    high_risk.append(a)
    low_risk.append(b)

import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='High Risk', x=list(df['sex'].unique()), y=high_risk),
    go.Bar(name='Low Risk', x=list(df['sex'].unique()), y=low_risk)
])

fig.update_layout(
    barmode='group',
    template='plotly_dark',  
    xaxis_title="<b>Sex</b>",
    yaxis_title="<b>Risk by Sex</b>",
    title='Distribution of <i>Risk Scores</i> by "Sex"',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    )
fig.show()

high_risk = []
low_risk = []

for element in list(df['age_cat'].unique()):
    a = df[df['age_cat'] == element]['label'].value_counts()[0]
    b = df[df['age_cat'] == element]['label'].value_counts()[1]
    high_risk.append(a)
    low_risk.append(b)

import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='High Risk', x=list(df['age_cat'].unique()), y=high_risk),
    go.Bar(name='Low Risk', x=list(df['age_cat'].unique()), y=low_risk)
])

fig.update_layout(
    barmode='group',
    template='plotly_dark',  
    xaxis_title="<b>Age</b>",
    yaxis_title="<b>Risk by Age</b>",
    title='Distribution of <i>Risk Scores</i> by "Age"',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    )
fig.show()

high_risk = []
low_risk = []

for element in list(df['race'].unique()):
    a = df[df['race'] == element]['label'].value_counts()[0]
    b = df[df['race'] == element]['label'].value_counts()[1]
    high_risk.append(a)
    low_risk.append(b)

import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='High Risk', x=list(df['race'].unique()), y=high_risk),
    go.Bar(name='Low Risk', x=list(df['race'].unique()), y=low_risk)
])

fig.update_layout(
    barmode='group',
    template='plotly_dark',  
    xaxis_title="<b>Race</b>",
    yaxis_title="<b>Risk by Race</b>",
    title='Distribution of <i>Risk Scores</i> by "Race"',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    )
fig.show()

**Let us now see if our sensitive attributes are correlated with `Risk`. If they are, this is already a signed that our future model could inherit these bias against a specific unprivileged class.**

**To be able to calculate correlations, let us transform all categorical values into numbers.**

In [10]:
corr_df = df.copy()

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
 
for column in list(set(df.columns) - set(df._get_numeric_data().columns)):
    corr_df[column] = le.fit_transform(corr_df[column])

import plotly.express as px

fig = px.imshow(corr_df.corr(numeric_only=True).values,
                labels=dict(x="Features", y="Features"),
                x=list(corr_df.columns),
                y=list(corr_df.columns),
                text_auto=True
                )
fig.update_xaxes(side='top')
fig.update_layout(template='plotly_dark',
                  title='Correlation Matrix',
                  coloraxis_showscale=False,
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')
fig.show()

**However, we can also speed up this vizualization process in our investigation by using tools like `pandas_profiling`.**

<img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/4c3692279382b860ef92ba7097363eefd6335d5a/68747470733a2f2f79646174612d70726f66696c696e672e79646174612e61692f646f63732f6173736574732f6c6f676f5f6865616465722e706e67" alt="drawing" width="600"/>

**[`Pandas-profiling`](https://pypi.org/project/pandas-profiling/) generates profile reports from a pandas `DataFrame`. Extending a pandas `DataFrame` with `df.profile_report()`, will automatically generate a standardized univariate and multivariate report for data understanding.**

**In this example, we will be using only 1000 samples to create this report.**

In [ ]:
from pandas_profiling import ProfileReport

profile = ProfileReport(df, title="Pandas Profiling Report")
profile.to_notebook_iframe()
profile.to_file("pandas_profiling_compas.html")

**According to the correlation scores, `race` has an alarming $0.22$ correlation with risk. Let us see how this will impact our future model.**

**To deal with the classification problem, we will create two classifiers: a `RandomForestClassifier` and a `LogisticRegressor`. We will make two classifiers to (1) compare their performance, and (2) because analysis of coefficients is not possible with forest-type classifiers (decision trees).**

In [11]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

seed = 42

X, y = df[df.columns.values.tolist()[0:12]], df[df.columns.values.tolist()[-1]]
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=seed
)

preprocess = make_column_transformer(
    (StandardScaler(), ['juv_fel_count', 'juv_misd_count', 
    'juv_other_count', 'priors_count', 'days_b_screening_arrest', 
    'c_days_from_compas']),
    (OneHotEncoder(), ['sex', 'age_cat', 'race', 'c_charge_degree', 
    'is_recid', 'is_violent_recid']))

from sklearn.ensemble import RandomForestClassifier

model_rf = make_pipeline(
    preprocess,
    RandomForestClassifier(max_depth=3, n_estimators=500))

model_rf.fit(X_train, y_train.values.ravel())

score = model_rf.score(X_test, y_test.values.ravel())

print(f'Accuracy (Random Forest): ' + '{:.2f}'.format(score * 100) + ' %')

Accuracy (Random Forest): 76.73 %


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import plotly.express as px

model_lr = make_pipeline(
    preprocess,
    LogisticRegression(penalty='l2', max_iter= 500))

model_lr.fit(X_train, y_train.values.ravel())

score = model_lr.score(X_test, y_test.values.ravel())

print(f'Accuracy (Logistic Regression): ' +
      '{:.2f}'.format(score * 100) + ' %')

preds = model_lr.predict(X_test)

matrix = confusion_matrix(y_test.values.ravel(), preds)

fig = px.imshow(matrix,
                labels=dict(x="Predicted", y="True label"),
                x=['High', 'Low'],
                y=['High', 'Low'],
                text_auto=True
                )
fig.update_xaxes(side='top')
fig.update_layout(template='plotly_dark',
                  title='Confusion Matrix (Logistic Regression Model)',
                  coloraxis_showscale=False,
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')
fig.show()

Accuracy (Logistic Regression): 80.08 %


**Accuracy varies considerably between classifiers. If we look at the confusion matrix of the second classifier, we see that the class that the algorithm has the most trouble getting right is "`High Risk`" (label = 0). While the algorithm's biggest error is in the False Negatives (individuals classified as "`Low`" who should be given "`High`" risk score).** 

**It is worth remembering that given the original distribution of the data (only 25% of samples are labeled "`High`") to achieve 75% accuracy on this problem, it would be sufficient for the classifier to label all entries as "`Low`".**

**Let's now analyze the _coefficients_ learned by the model during its training.**

In [4]:
coefs = pd.DataFrame(
    model_lr[-1].coef_,
    columns=model_lr[:-1].get_feature_names_out(),
    index=['Coefficients']).transpose()

display(coefs)
import plotly.graph_objects as go

fig = go.Figure(go.Bar(
    x=coefs['Coefficients'],
    y=model_lr[:-1].get_feature_names_out(),
    orientation='h'))
fig.update_xaxes(range=[model_lr[-1].coef_.min(
) + (model_lr[-1].coef_.min() * 0.1), model_lr[-1].coef_.max() + (model_lr[-1].coef_.max() * 0.1)])
fig.update_layout(
    xaxis=dict(
        tickmode='linear',
        tick0=0,
        dtick=0.5
    ),
    template='plotly_dark',
    title_text='LogisticRegression Coefficients',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)'

)
fig.show()

,Coefficients
standardscaler__juv_fel_count,-0.256862
standardscaler__juv_misd_count,-0.156000
standardscaler__juv_other_count,-0.120214
standardscaler__priors_count,-0.779870
standardscaler__days_b_screening_arrest,-0.040276
standardscaler__c_days_from_compas,-0.111659
onehotencoder__sex_Female,0.063771
onehotencoder__sex_Male,-0.039929
onehotencoder__age_cat_25 - 45,-0.079381
onehotencoder__age_cat_Greater than 45,1.108571


**At first glance, several things seem to contribute equally to this classifier.**

**However, we need to be cautious when interpreting coefficients from linear models. Since each feature represents a _measured quantity on its own scale_, it doesn't make sense to compare them:**

- **E.g., age can range from, e.g.,  16 to 100, but binary features only from 0 to 1. _This does not mean that an age of 100 has 100 times more weight than a feature with gender_.**

**To get a more correct view, we first need to normalize these values by their `standard deviation` (something that brings all the values to a common scale).**

In [7]:
X_train_preprocessed = pd.DataFrame(
    model_lr[:-1].transform(X_train), columns=model_lr[:-1].get_feature_names_out(),
)

coefs = pd.DataFrame(
    model_lr[-1].coef_ * X_train_preprocessed.std(axis=0).values,
    columns=model_lr[:-1].get_feature_names_out(),
    index=['Coefficients (Normalized by STD)']).transpose()

display(coefs)

fig = go.Figure(go.Bar(
    x=coefs['Coefficients (Normalized by STD)'],
    y=model_lr[:-1].get_feature_names_out(),
    orientation='h'))
fig.update_xaxes(range=[model_lr[-1].coef_.min(
) + (model_lr[-1].coef_.min() * 0.1), model_lr[-1].coef_.max() + (model_lr[-1].coef_.max() * 0.1)])
fig.update_layout(
    xaxis=dict(
        tickmode='linear',
        tick0=0,
        dtick=0.5
    ),
    template='plotly_dark',
    title_text='LogisticRegression Coefficients (Normalized by STD)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)'

)
fig.show()

,Coefficients (Normalized by STD)
standardscaler__juv_fel_count,-0.256872
standardscaler__juv_misd_count,-0.156006
standardscaler__juv_other_count,-0.120218
standardscaler__priors_count,-0.779899
standardscaler__days_b_screening_arrest,-0.040277
standardscaler__c_days_from_compas,-0.111663
onehotencoder__sex_Female,0.024641
onehotencoder__sex_Male,-0.015429
onehotencoder__age_cat_25 - 45,-0.039286
onehotencoder__age_cat_Greater than 45,0.439112


**Now we have a more reliable view of what our model has learned. The most important features for this problem are `race`, `age` and `prior`.**

**As [Cynthia Rudin](https://www.nature.com/articles/s42256-019-0048-x#auth-Cynthia-Rudin) puts: we shuold _[stop explaining black box machine learning models for high stakes decisions and use interpretable models instead](https://www.nature.com/articles/s42256-019-0048-x)_.**

**For example, a fully interpretable model could be used in this problem. A model that can be written in 4 lines of pseudocode:**

````

If sample has < 25 years & is African-American & has prior counts > 0:
	Risk = High
Else:
	Risk = Low
	
````

**Lets call this model the `evil_model` and see how it performs incomparisson to our `RandomForestClassifier`.**

In [13]:
def evil_model(df):

    predictions = []

    for i in range(len(df)):

        sample = df.iloc[i]

        if sample['age_cat'] == 'Less than 25' and \
            sample['race'] == 'African-American' and \
                sample['priors_count'] > 0:

            predictions.append(0)

        else:

            predictions.append(1)
            
    return predictions

predictions = evil_model(X_test)

matrix = confusion_matrix(y_test, predictions)
TN, FP, FN, TP = matrix.ravel()

print(f'Accuracy (Evil-Model): {round(((TP + TN)/(TP + FP + TN + FN)) * 100, 2)}%')


fig = px.imshow(matrix,
                labels=dict(x="Predicted", y="True label"),
                x=['High', 'Low'],
                y=['High', 'Low'],
                text_auto=True
                )
fig.update_xaxes(side='top')
fig.update_layout(template='plotly_dark',
                  title='Confusion Matrix (Evil_Model)',
                  coloraxis_showscale=False,
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')
fig.show()


Accuracy (Evil-Model): 74.65%


- **Accuracy (Random Forest): 76.56 %;**
- **Accuracy (Evil-Model): 74.65%.**

**The `evil_model` even seems to be wrong in the same way that the `LogisticRegressor` (i.e., most of the mistakes are False Neagtives).**

**The big difference between this model (`evil_model`) and the model produced by Northpointe is that the Northpointe model is a black-box. We simply do not know how it works. Whereas in the case of the `evil_model`, with similar accuracy to the original model, we know exactly what it does. And so, we can say: _this is unacceptable_.**

In [14]:
from IPython.display import Markdown

def calc_fair(model, DataFrame, protected_atributte, group_priv, group_unpriv, label):
    test_set = DataFrame

    test_set_priv_labels, test_set_priv = list(test_set[test_set[protected_atributte] == group_priv][label]), test_set[test_set[protected_atributte] == group_priv].drop(label, axis = 1)
    test_set_unpriv_labels, test_set_unpriv = list(test_set[test_set[protected_atributte] == group_unpriv][label]), test_set[test_set[protected_atributte] == group_unpriv].drop(label, axis = 1)
    
    preds_priv = model.predict(test_set_priv)
    preds_unpriv = model.predict(test_set_unpriv)

    TN_PV, FP_PV, FN_PV, TP_PV = confusion_matrix(test_set_priv_labels, preds_priv).ravel()
    TN_UPV, FP_UPV, FN_UPV, TP_UPV = confusion_matrix(test_set_unpriv_labels, preds_unpriv).ravel()

    statistical_parity_priv = (TP_PV + FP_PV)/(TP_PV + FP_PV + TN_PV + FN_PV)  # STATISTICAL PARITY RATIO
    statistical_parity_unpriv = (TP_UPV + FP_UPV)/(TP_UPV + FP_UPV + TN_UPV + FN_UPV)  # STATISTICAL PARITY RATIO
    equal_oportunity_priv = TP_PV / (TP_PV+FN_PV)  # TRUE POSITIVE RATIO
    equal_oportunity_unpriv = TP_UPV / (TP_UPV+FN_UPV)  # TRUE POSITIVE RATIO
    predictive_parity_priv = TP_PV/(TP_PV + FP_PV)  # POSITIVE PREDICTIVE VALUE
    predictive_parity_unpriv = TP_UPV/(TP_UPV + FP_UPV)  # POSITIVE PREDICTIVE VALUE
    predictive_equality_priv = FP_PV / (FP_PV+TN_PV)  # FALSE POSITIVE RATE
    predictive_equality_unpriv = FP_UPV / (FP_UPV+TN_UPV)  # FALSE POSITIVE RATE
    accuracy_equality_priv = (TP_PV + TN_PV)/(TP_PV + FP_PV + TN_PV + FN_PV)  # ACCURACY EQUALITY RATIO
    accuracy_equality_unpriv = (TP_UPV + TN_UPV)/(TP_UPV + FP_UPV + TN_UPV + FN_UPV)  # ACCURACY EQUALITY RATIO 

    if statistical_parity_priv >= statistical_parity_unpriv:
        statistical_parity_ratio = statistical_parity_unpriv/statistical_parity_priv
    elif statistical_parity_priv < statistical_parity_unpriv:
        statistical_parity_ratio = statistical_parity_priv/statistical_parity_unpriv

    if equal_oportunity_priv >= equal_oportunity_unpriv:
        equal_oportunity_ratio = equal_oportunity_unpriv/equal_oportunity_priv
    elif equal_oportunity_priv < equal_oportunity_unpriv:
        equal_oportunity_ratio = equal_oportunity_priv/equal_oportunity_unpriv
    
    if predictive_parity_priv >= predictive_parity_unpriv:
        predictive_parity_ratio = predictive_parity_unpriv/predictive_parity_priv
    elif predictive_parity_priv < predictive_parity_unpriv:
        predictive_parity_ratio = predictive_parity_priv/predictive_parity_unpriv

    if predictive_equality_priv >= predictive_equality_unpriv:
        predictive_equality_ratio = predictive_equality_unpriv/predictive_equality_priv
    elif predictive_equality_priv < predictive_equality_unpriv:
        predictive_equality_ratio = predictive_equality_priv/predictive_equality_unpriv
    
    if accuracy_equality_priv >= accuracy_equality_unpriv:
        accuracy_equality_ratio = accuracy_equality_unpriv/accuracy_equality_priv
    elif accuracy_equality_priv < accuracy_equality_unpriv:
        accuracy_equality_ratio = accuracy_equality_priv/accuracy_equality_unpriv

    equalized_odds = f'TPR: {round(equal_oportunity_priv, 2)} vs {round(equal_oportunity_unpriv, 2)} <br> FPR: {round(predictive_equality_priv,2)} vs {round(predictive_equality_unpriv,2)}'

    data = {'Fairness Metrics': ['Chance of receiving the positive class - privileged',
                                'Chance of receiving the positive class - unprivileged',
                                'Statistical Parity Ratio (SPR)',
                                'True Positive Rate - privileged',
                                'True Positive Rate - unprivileged',
                                'Equal Opportunity Ratio (EOR)',
                                'Positive Predictive Value - privileged',
                                'Positive Predictive Value - unprivileged',
                                'Predictive Parity Ratio (PPR)',
                                'False Positive Rate - privileged',
                                'False Positive Rate - unprivileged',
                                'Predictive Equality Ratio (PER)',
                                'Accuracy - privileged',
                                'Accuracy - unprivileged',
                                'Accuracy Equality Ratio (AER)',
                                'Equalized Odds'],
            'Scores': [round(statistical_parity_priv, 2),
                        round(statistical_parity_unpriv, 2),
                        round(statistical_parity_ratio,2),
                        round(equal_oportunity_priv, 2),
                        round(equal_oportunity_unpriv, 2),
                        round(equal_oportunity_ratio, 2),
                        round(predictive_parity_priv,2),
                        round(predictive_parity_unpriv,2),
                        round(predictive_parity_ratio,2),
                        round(predictive_equality_priv,2),
                        round(predictive_equality_unpriv,2),
                        round(predictive_equality_ratio,2),
                        round(accuracy_equality_priv,2),
                        round(accuracy_equality_unpriv,2),
                        round(accuracy_equality_ratio,2),
                        f'TPR: {round(equal_oportunity_priv, 2)} vs {round(equal_oportunity_unpriv, 2)}. FPR: {round(predictive_equality_priv,2)} vs {round(predictive_equality_unpriv,2)}']
            }
    df = pd.DataFrame(data).set_index('Fairness Metrics')

    return pd.DataFrame(data).set_index('Fairness Metrics')

X_test['labels'] = y_test

fairness_df = calc_fair(model_lr, X_test, 'race', 'Caucasian', 'African-American', 'labels')
display(Markdown(fairness_df.to_markdown()))

| Fairness Metrics                                      | Scores                               |
|:------------------------------------------------------|:-------------------------------------|
| Chance of receiving the positive class - privileged   | 0.96                                 |
| Chance of receiving the positive class - unprivileged | 0.8                                  |
| Statistical Parity Ratio (SPR)                        | 0.83                                 |
| True Positive Rate - privileged                       | 0.99                                 |
| True Positive Rate - unprivileged                     | 0.91                                 |
| Equal Opportunity Ratio (EOR)                         | 0.93                                 |
| Positive Predictive Value - privileged                | 0.86                                 |
| Positive Predictive Value - unprivileged              | 0.75                                 |
| Predictive Parity Ratio (PPR)                         | 0.87                                 |
| False Positive Rate - privileged                      | 0.84                                 |
| False Positive Rate - unprivileged                    | 0.58                                 |
| Predictive Equality Ratio (PER)                       | 0.7                                  |
| Accuracy - privileged                                 | 0.86                                 |
| Accuracy - unprivileged                               | 0.74                                 |
| Accuracy Equality Ratio (AER)                         | 0.87                                 |
| Equalized Odds                                        | TPR: 0.99 vs 0.91. FPR: 0.84 vs 0.58 |

**Above we can see the Fairness evaluation of the best model (`LogisticRegressor`). To learn more about ML Fairness, and how these scores are computed visit our directory on [ML Fairness](https://github.com/Nkluge-correa/teeny-tiny_castle/tree/master/ML%20Fairness).**

**Now let's create an `Explainer` around our model using the [Dalex](https://dalex.drwhy.ai/python/) library.**

In [15]:
import dalex as dx

model_lr_exp = dx.Explainer(model_lr,
                            X, y, label='Logistic Regression explainer',
                            model_type='binary classification')

Preparation of a new explainer is initiated

  -> data              : 17019 rows 12 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 17019 values
  -> model_class       : sklearn.linear_model._logistic.LogisticRegression (default)
  -> label             : Logistic Regression explainer
  -> predict function  : <function yhat_proba_default at 0x000001CF8EC2D820> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 1.87e-05, mean = 0.743, max = 0.995
  -> model type        : binary classification will be used
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.991, mean = 0.000594, max = 0.997
  -> model_info        : package sklearn

A new explainer has been created!


**To finish our interpretability analysis, we will set up a Break-Down plot of two different samples. One of the samples has the sensitive attribute "`African-American`", and the other has the attribute "`Caucasian`".**

In [16]:
print('Probabilities for sample_1 (African-American): ', \
    model_lr.predict_proba(pd.DataFrame(df[df['label'] == 0]\
        .drop('label', axis = 1).iloc[27]).transpose()))

display(df[df['label'] == 0].drop('label', axis = 1).iloc[27])

print('Probabilities for sample_1 (Caucasian): ', \
    model_lr.predict_proba(pd.DataFrame(df[df['label'] == 1]\
        .drop('label', axis = 1).iloc[14]).transpose()))
        
display( df[df['label'] == 1].drop('label', axis = 1).iloc[14])


Probabilities for sample_1 (African-American):  [[0.96979264 0.03020736]]


sex                                    Male
age_cat                             25 - 45
race                       African-American
juv_fel_count                             2
juv_misd_count                            1
juv_other_count                           3
priors_count                             21
days_b_screening_arrest                 0.0
c_days_from_compas                      0.0
c_charge_degree                        (F2)
is_recid                                  1
is_violent_recid                          0
Name: 101, dtype: object

Probabilities for sample_1 (Caucasian):  [[0.04983306 0.95016694]]


sex                           Female
age_cat                      25 - 45
race                       Caucasian
juv_fel_count                      0
juv_misd_count                     0
juv_other_count                    0
priors_count                       0
days_b_screening_arrest         -1.0
c_days_from_compas               1.0
c_charge_degree                 (M1)
is_recid                           0
is_violent_recid                   0
Name: 17, dtype: object

In [17]:
sample_african_american = df[df['label'] == 0].drop('label', axis = 1).iloc[27]

bd_sample_1 = model_lr_exp.predict_parts(sample_african_american,
                                         type='break_down')

fig = bd_sample_1.plot(show=False)

fig.update_layout(
    template='plotly_dark',
    title='sample_african_american (Logistic Regression Explainer)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    font_color='white')
fig.show()

sample_caucasian = df[df['label'] == 1].drop('label', axis = 1).iloc[14]

bd_sample_2 = model_lr_exp.predict_parts(sample_caucasian,
                                         type='break_down')

fig = bd_sample_2.plot(show=False)

fig.update_layout(
    template='plotly_dark',
    title='sample_caucasian (Logistic Regression Explainer)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    font_color='white')
fig.show()

**XAI and ML Fairness bring us new tools to investigate the social ills that are embedded in our societies.**

**Tools used in ML Fairness and XAI can help us better understand how AI systems work. With this knowledge, we can take several directions of action, such as creating interpretable models, or deciding that a model should not be used for a particular application.** ⚖️🔎

----

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).